# Implementing Backpropagation

This is an exercise from Udacity Nanodegree **nd101-ent** lesson 2.8

## Data 

In [1]:
import warnings; warnings.simplefilter('ignore')
import numpy as np
import pandas as pd

admissions = pd.read_csv('data/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

We have 6 features:

In [2]:
features

,gre,gpa,rank_1,rank_2,rank_3,rank_4
106,0.972155,0.446965,1,0,0,0
9,0.972155,1.392922,0,1,0,0
61,-0.239793,-0.183673,0,0,0,1
224,1.837832,-1.287291,0,1,0,0
37,-0.586063,-1.287291,0,0,1,0
...,...,...,...,...,...,...
28,1.664697,-0.446439,0,1,0,0
346,0.106478,-0.104844,0,1,0,0
271,-0.412928,-0.157397,0,0,1,0
338,-0.412928,-0.498993,1,0,0,0


## Model Architecture

![model](images/2-8_nn.png)

## Implementation of Backpropagation

In [3]:
np.random.seed(21)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# ----- Hyperparameters -----------------------------
n_hidden= 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5, size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5, size=n_hidden)

In [4]:
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        
        ## forward pass ##
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## backward pass ##
        error = y - output
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer

        # the hidden layer's contribution to the error:
        hidden_error = output_error_term * hidden_output
        
        # the error term for the hidden layer:
        hidden_error_term = np.dot(output_error_term, weights_input_hidden) * hidden_output * (1 - hidden_output)
        
        # update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.25135784720177
Train loss:  0.25001697258522415
Train loss:  0.248717824809504
Train loss:  0.24745956973223776
Train loss:  0.2462413545575023
Train loss:  0.24506231119263724
Train loss:  0.24392155942779803
Train loss:  0.24281820993418365
Train loss:  0.24175136707873493
Train loss:  0.2407201315547637
Prediction accuracy: 0.725


## Notes

There are two things to mention:

1. The calculation of the weights change of the hidden layer `del_w_input_hidden += hidden_error_term * x[:,None]`, we do an **element wise** multiplication of a matrix `hidden_error_term` with the vector `x`, hence must transform the vector to a a **2D-vector**:

In [5]:
x.shape

(6,)

In [6]:
x[:, None].shape

(6, 1)

In [7]:
hidden_error_term * x[:,None]

array([[-0.00070294, -0.0014713 ],
       [ 0.01803146, -0.02124858],
       [-0.        ,  0.        ],
       [ 0.00209494,  0.00233943],
       [-0.        ,  0.        ],
       [ 0.        , -0.        ]])

2. We sum the weights change $\Delta w$ for each record (feature) from the data set. We therefore must **divide by the number of records** before updating the weights

In [8]:
print(del_w_input_hidden)
print(n_records)

[[-0.03660627 -0.07227949]
 [ 0.32192302 -0.2694654 ]
 [-0.04282539  0.11350785]
 [ 0.10634632  0.1189796 ]
 [-0.81061487  0.00931435]
 [ 0.27525864 -0.56672298]]
360


In [9]:
del_w_input_hidden / n_records

array([[-1.01684089e-04, -2.00776351e-04],
       [ 8.94230613e-04, -7.48514989e-04],
       [-1.18959405e-04,  3.15299591e-04],
       [ 2.95406454e-04,  3.30498894e-04],
       [-2.25170798e-03,  2.58731962e-05],
       [ 7.64607326e-04, -1.57423050e-03]])